In [1]:
import json, ast, re
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

### Utility Functions

In [2]:
def load_items(filepath):
    print("Loading %s ..." % filepath)
    lines = open(filepath,"r").readlines()
    items = []
    category_map = {}
    for i, line in enumerate(lines):
        line = line.strip()
        if i == 0:
            task_name = line
        elif len(line) > 0:
            parts = line.split(":",1)
            category = parts[0].strip()
            for item in parts[1].strip().split(";"):
                if len(item) > 0:
                    items.append(item)
                    category_map[item] = category
    return task_name,items,category_map

In [3]:
def get_dict_data(filepath):
    print("Loading %s ..." % filepath)
    dict_data = json.load(open(filepath))
    dict_data = {ast.literal_eval(k): v for k, v in dict_data.items()}
    #dict_data = {frozenset(k): v for k, v in dict_data.items()}
    return dict_data

In [4]:
def create_similarity_matrix( items, dict_data ):
    n = len(items)
    S = np.zeros([n,n])
    index_map = {}
    for ind, item in enumerate(items):
        index_map[item] = ind
    for pair in dict_data:
        ind1 = index_map[pair[0]]
        ind2 = index_map[pair[1]]
        sim = dict_data[pair]
        S[ind1,ind2] = sim
        S[ind2,ind1] = sim
    return S

In [5]:
def create_distance_matrix( items, filepath ):
    sim_dict = get_dict_data( filepath )
    S = create_similarity_matrix( items, sim_dict )
    D = 1-(S/S.max())
    return D

In [6]:
# From sklearn calinski_harabaz_score, but need dispersion separately:
def between_within_dispersion(X, labels, cluster_names):
    """
    Based on
    .. [1] `T. Calinski and J. Harabasz, 1974. "A dendrite method for cluster
       analysis". Communications in Statistics
       <http://www.tandfonline.com/doi/abs/10.1080/03610927408827101>`_
    """    
    n_samples, _ = X.shape
    n_labels = max(labels)+1
    mean = np.mean(X, axis=0)
    extra_disp = 0.0
    intra_disp = 0.0
    results = pd.DataFrame(columns=["Categoey","Size","Within-SS", "Between-SS"])
    for k in range(n_labels):
        cluster_k = X[labels == k]
        mean_k = np.mean(cluster_k, axis=0)
        bgss = len(cluster_k) * np.sum((mean_k - mean) ** 2)
        wgss = np.sum((cluster_k - mean_k) ** 2)
        extra_disp += bgss
        intra_disp += wgss        
        results.loc[k] = (cluster_names[k], len(cluster_k), np.round(wgss,2), np.round(bgss,2)) 
    mean_within = wgss/k
    mean_between = bgss/k
    ch = extra_disp * (n_samples - n_labels) / (intra_disp * (n_labels - 1.))    
    return (results,ch,mean_within,mean_between)

### Perform the Validation

In [7]:
methods = ["eve","word2vec_cbow","word2vec_sg","fasttext_cbow","fasttext_sg","glove"]
dataset_ids = ["european_cities", "movie_genres", "animal_classes", "cuisine", "music_genres", "nobel_laureates",
               "country_continent"]

In [8]:
result_cols = ["Dataset","Method","Mean Within","Mean Between","CH-Index"]
result_rows = []
for dataset_id in dataset_ids:
    # Load the data
    print()
    task_name,items,category_map = load_items("../dataset/tasks/%s.txt" % dataset_id )
    categories = list(set(category_map.values()))
    categories.sort()
    print("Task: %s" % task_name)
    print("%d categories: %s" % (len(categories),categories) )
    # Convert to labels
    labels = []
    for item in items:
        labels.append( categories.index( category_map[item] ) )
    labels = np.array(labels)    
    # Create distance matrices
    print("Creating distance matrices ...")
    D = {}
    for method_id in methods:
        filepath = "../output/pairwise_similarity/%s_%s.json" % (dataset_id,method_id) 
        D[method_id] = create_distance_matrix(items, filepath )
    # Calculate the scores
    print("Validating by measures ...")
    for method_id in methods:
        results,ch,mean_within,mean_between = between_within_dispersion( D[method_id], labels, categories )
        result_rows.append( [dataset_id, method_id, mean_within, mean_between, ch ] )


Loading ../dataset/tasks/european_cities.txt ...
Task: European cities
5 categories: ['France', 'Germany', 'Great Britain', 'Italy', 'Spain']
Creating distance matrices ...
Loading ../output/pairwise_similarity/european_cities_eve.json ...
Loading ../output/pairwise_similarity/european_cities_word2vec_cbow.json ...
Loading ../output/pairwise_similarity/european_cities_word2vec_sg.json ...
Loading ../output/pairwise_similarity/european_cities_fasttext_cbow.json ...
Loading ../output/pairwise_similarity/european_cities_fasttext_sg.json ...
Loading ../output/pairwise_similarity/european_cities_glove.json ...
Validating by measures ...

Loading ../dataset/tasks/movie_genres.txt ...
Task: Movie genres
5 categories: ['Animation', 'Crime film', 'Horror film', 'Science fiction film', 'Western (genre)']
Creating distance matrices ...
Loading ../output/pairwise_similarity/movie_genres_eve.json ...
Loading ../output/pairwise_similarity/movie_genres_word2vec_cbow.json ...
Loading ../output/pairwi

In [9]:
df_results = pd.DataFrame( result_rows, columns = result_cols )
df_results = df_results.round(3)
df_results

,Dataset,Method,Mean Within,Mean Between,CH-Index
0,european_cities,eve,3.126,2.391,29.081
1,european_cities,word2vec_cbow,7.715,17.141,48.572
2,european_cities,word2vec_sg,5.455,7.448,28.980
3,european_cities,fasttext_cbow,8.303,13.244,33.725
4,european_cities,fasttext_sg,5.750,10.857,41.884
5,european_cities,glove,6.859,3.835,15.534
6,movie_genres,eve,6.922,1.582,12.447
7,movie_genres,word2vec_cbow,11.982,0.396,1.358
8,movie_genres,word2vec_sg,6.044,0.184,1.431
9,movie_genres,fasttext_cbow,9.807,0.408,1.509


In [10]:
df_results.to_csv("cluster-validation.csv")